# Module 1: Introduction to Exploratory Analysis 

What we'll be doing in this notebook:
-----

 1.  Checking variable type
 2.  Checking for missing variables 
 3.  Number of observations in the dataset
 4.  Descriptive statistics

### Import packages

In [52]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import dateutil.parser

# The command below means that the output of multiple commands in a cell will be output at once
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# The command below tells jupyter to display up to 80 columns, this keeps everything visible
pd.set_option('display.max_columns', 80)
pd.set_option('expand_frame_repr', True)

# Show figures in notebook
%matplotlib inline

### Import dataset

We read in our dataset

In [53]:
path = '/home/catherine/Desktop/'
filename = 'DS_B/data/loans.csv'

df = pd.read_csv(path+filename)

In the cell below, we take a random sample of 2 rows to get a feel for the data.

In [54]:
df.sample(n=2)

,id_number,loan_amount,lender_count,status,funded_date,funded_amount,repayment_term,location_country_code,sector,description,use
5502,1484334,1450,32,funded,2018-03-09T03:09:53Z,1450,9,ZM,Services,Thandiwe is very excited to be getting her fou...,to increase the float in several of Thandiwe's...
2867,1512505,4125,89,funded,2018-05-01T09:04:34Z,4125,9,ML,Agriculture,Niama is the one holding onto the sheep. She i...,to buy sheep for resale.


### 1) Type Checking
<a id='type_check'></a>

Type is very important in Python programing, because it affects the types of functions you can apply to a series. There are a few different types of data you will see regularly (see [this](https://en.wikibooks.org/wiki/Python_Programming/Data_Types) link for more detail):
* **int** - a number with no decimal places. example: loan_amount field
* **float** - a number with decimal places. example: partner_id field
* **str** - str is short for string. This type formally defined as a sequence of unicode characters. More simply, string means that the data is treated as a word, not a number. example: sector
* **boolean** - can only be True or False. There is not currently an example in the data, but we will be creating a gender field shortly.
* **datetime** - values meant to hold time data. Example: posted_date

Let's check the type of our variables using the examples we saw in the cell above.

In [55]:
# Here are all of the columns
df.columns.tolist()

['id_number',
 'loan_amount',
 'lender_count',
 'status',
 'funded_date',
 'funded_amount',
 'repayment_term',
 'location_country_code',
 'sector',
 'description',
 'use']

In [56]:
# Find the dtype, aka datatype, for a column
df['id_number'].dtype

dtype('int64')

In [57]:
# Try this - Pick a couple of columns and check their type on your own


### 2) Do I have missing values?

<a id='missing_check'></a>

Missing data in the training data set can reduce the power / fit of a model or can lead to a biased model because we have not analysed the behavior and relationship with other variables correctly. It can lead to wrong prediction or classification.

If we have missing data, is the missing data at random or not? If data is missing at random, the data distribution is still representative of the population. You can probably ignore the missing values as an inconvenience. However, if the data is systematically missing, the analysis you do may be biased. You should carefully consider the best way to clean the data, it may involve dropping some data.

We want to see how many values are missing in certain variable columns. One way to do this is to count the number of null observations. 

For this, we wrote a short function to apply to the dataframe. 

We print out the first few observations, but you can remove the .head() to print out all columns. 

In [58]:
#Create a new function:
def num_missing(x):
    return sum(x.isnull())

#Applying per column:
print("Missing values per column:")
## Check how many are missing by column, and then check which ones have any missing values
print(df.apply(num_missing, axis=0).where(lambda x : x != 0).dropna()) 
#axis=0 defines that function is to be applied on each column

Missing values per column:
funded_date              937.0
location_country_code     17.0
description              342.0
use                      342.0
dtype: float64


In [59]:
#checking the % of missing values
total = df.isnull().sum().sort_values(ascending = False)

percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending = False)

missing_kiva_loans_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_kiva_loans_data

,Total,Percent
funded_date,937,0.155674
use,342,0.056820
description,342,0.056820
location_country_code,17,0.002824
sector,0,0.000000
repayment_term,0,0.000000
funded_amount,0,0.000000
status,0,0.000000
lender_count,0,0.000000
loan_amount,0,0.000000


### 3) Sanity Checks
<a id='obs_check'></a>

**Does the dataset match what you expected to find?**
- is the range of values what you would expect. For example, are all loan_amounts above 0.
- do you have the number of rows you would expect
- is your data for the date range what you would expect. For example, is there a strange year in the data like 1880.
- are there unexpected spikes when you plot the data over time


In the command below we find out the number of rows and number of columns by using the function shape. You can also use len(df.index) to find the number of rows.

In [60]:
print(f'There are {df.shape[0]} observations and {df.shape[1]} features')

There are 6019 observations and 11 features


Remember, each row is an observation and each column is a potential feature. 

Remember we need a large amount of data for machine learning.

### 4) Descriptive statistics of the dataset

<a id='desc_stats'></a>

In [61]:
#Try out - Write code that provides key summary statistics for each numeric column.

In order to get the same summary statistics for categorical columns (string) we need to do a little data wrangling. The first line of code filters for all columns that are a data type object. As we know from before this means they are considered to be a string. The final row of code provides summary statistics for these character fields.

In [62]:
categorical = df.dtypes[df.dtypes == "object"].index
df[categorical].describe()

,status,funded_date,location_country_code,sector,description,use
count,6019,5082,6002,6019,5677,5677
unique,3,4453,30,14,5277,4325
top,funded,2018-07-22T15:54:41Z,TZ,Food,With the income produced during the previous l...,to pay for a stove.
freq,5082,9,400,1738,2,80


In [63]:
#Try out - What's the other way one can obtain the information above?

In the table above, there are 4 really useful fields: 

1) **count** - total number of fields populated (Not empty). 

2) **unique** - tells us how many different unique ways this field is populated.

3) **top** - tells us the most popular data point.

4) **freq** - tells us that how frequent the most popular category is in our dataset. 

What is next
-----

In the next section, we move on to exploratory data analysis (EDA).

<br>
<br> 
<br>

----